In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import numpy as np
from dark_emulator_public import dark_emulator
import os, sys, time, json
import matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.interpolate import InterpolatedUnivariateSpline as ius
from scipy.interpolate import interp2d, interp1d
from scipy.integrate import simps
from tqdm import tqdm
import hsc3x2pt

using dark_emulator at  /lustre/work/sunao.sugiyama/package/dark_emulator_public/dark_emulator/__init__.py


# パワースペクトルclass
中でdark emulator, halofitを動かしている。

In [5]:
power_b1 = hsc3x2pt.power_b1_class()

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


# define source sample

In [6]:
samplename = 's1'
dzph1  = 0   # photoz bias
dm1    = 0   # multiplicative bias
A_IA   = 0   # intrinsic alignment
eta_IA = 0   # IA redshift dependence
sigma_e= 0.2 # shape noise
nsource= '4.75 arcmin^-2' # number density of source galaxy
fname_Nz = 'sourcePzs_Y1/MLZs3.txt'

g_s = hsc3x2pt.galaxy_sample_source_IA_class([samplename, fname_Nz, dzph1, dm1, A_IA, eta_IA, sigma_e, nsource])

# C(l) class
powerをclに変換すすclass。

In [7]:
pk2cl = hsc3x2pt.pk2cl_class(power_b1)

In [8]:
# source sampleをセットする
pk2cl.set_galaxy_sample(g_s)

In [11]:
# 宇宙論parameterを与える。dark emulatorへのinput
cosmo_dict = dict(zip(['omega_b', 'omega_c', 'Omega_de', 'ln10p10As', 'n_s', 'w_de'], 
                      [0.02225, 0.1198, 0.6844, 3.094, 0.9645, -1]))
pk2cl.set_cosmology_from_dict(cosmo_dict)

In [12]:
#与えた宇宙論parameterで、power spectrumを計算する。
pk2cl.init_pk()

In [13]:
# lを定義
l = np.logspace(-2, 5, 1000)

In [15]:
# C(l)を計算。
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l)

:4.9214818477630615 sec


In [16]:
# save C(l)
#dirname_to_save = 'test'
#pk2cl.dump_Cl_cache(dirname_to_save, overwrite=True, silent=True)

In [19]:
pk2cl.Cl_cache['Cl'].keys()

odict_keys(['l', 's1,s1'])